# Data Processing for UI

 1. grid_id (Grid ID)
 2. central_lat (Central Latitude)
 3. central_lon (Central Longitude)
 4. total_event_count (Total Event Count)
 5. total_flood_event (Total Flood Events)
 6. earliest_event_year (Earliest Event Year)
 7. latest_event_year (Latest Event Year)
 8. total_damages (Total Damages)
 9. primary_admin1 (Primary Administrative Region)
 10. event_summary (Event Summary)

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

# %cd drive/My Drive/mml_flood/

Mounted at /content/drive/
/content/drive/My Drive/mml_flood


### Read data

In [9]:
#gdis data - geocoded disasters
gdis = pd.read_csv('../data/disaster/pend-gdis-1960-2018-disasterlocations.csv')
# #emdat data - international disasters
emdat = pd.read_csv('../data/disaster/emdat_public_2022_09_21_query_uid-47Yzpr.csv', skiprows=[0,1,2,3,4,5])

/var/folders/4p/g0zxwnsd5lvdf5d6rvd07rvr0000gp/T/ipykernel_43960/2850920651.py:4: DtypeWarning: Columns (8,16,17,18,19,24,25,26,27,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  emdat = pd.read_csv('../data/disaster/emdat_public_2022_09_21_query_uid-47Yzpr.csv', skiprows=[0,1,2,3,4,5])


  ### Filter emdat columns

In [10]:
emdat['disasterno'] = emdat['Dis No'].str[:-4]  # Remove last 4 characters "-XYZ" area code

In [12]:
emdat_cols = ['disasterno', 'Year', 'Event Name',
              'Start Year', 'Start Month', 'Start Day',
              'End Year', 'End Month', 'End Day',
              "Total Damages, Adjusted ('000 US$)",
              'Total Deaths', 'Total Affected',
              "Reconstruction Costs, Adjusted ('000 US$)",
              'Disaster Subtype', 'OFDA Response', 'River Basin']

# Subset EMDAT to available columns (ignore missing ones if not present)
emdat = emdat[[col for col in emdat_cols if col in emdat.columns]]
print(emdat['OFDA Response'].count())  # check available data
print(emdat['River Basin'].count())    # check available data

1716
1312


In [13]:
print(emdat.columns)

Index(['disasterno', 'Year', 'Event Name', 'Start Year', 'Start Month',
       'Start Day', 'End Year', 'End Month', 'End Day',
       'Total Damages, Adjusted ('000 US$)', 'Total Deaths', 'Total Affected',
       'Reconstruction Costs, Adjusted ('000 US$)', 'Disaster Subtype',
       'OFDA Response', 'River Basin'],
      dtype='object')


In [14]:
print(gdis.columns)

Index(['id', 'country', 'iso3', 'gwno', 'year', 'geo_id', 'geolocation',
       'level', 'adm1', 'adm2', 'adm3', 'location', 'historical',
       'hist_country', 'disastertype', 'disasterno', 'latitude', 'longitude'],
      dtype='object')


### Merged cleaned emdat with gdis on disasterno

In [16]:
merged_df = pd.merge(emdat, gdis, on='disasterno', how='right')
merged_df.shape

(82885, 33)

In [17]:
# Drop duplicate records based on GDIS 'id'
merged_df = merged_df.drop_duplicates(subset=['id'])
print("Merged Data Shape:", merged_df.shape)

Merged Data Shape: (9924, 33)


### Create a look up table including all disasters over all locations, lat and lon, location names

In [ ]:
#create an update table for all disasters 
df_all = merged_df.copy()

# Create grid_id from GDIS coordinates
df_all['lat'] = df_all['latitude'].round().astype(int)
df_all['lon'] = df_all['longitude'].round().astype(int)
df_all['grid_id'] = df_all['lat'].astype(str) + "_" + df_all['lon'].astype(str)

# Aggregate all params
agg_all = df_all.groupby('grid_id').agg(
    total_disaster_count = ('disasterno', 'count'),
    total_damages = ("Total Damages, Adjusted ('000 US$)", 'sum'),
    lat = ('lat', 'first'),
    lon = ('lon', 'first'),
    primary_admin1 = ('adm1', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan),
    unique_locations = ('location', lambda x: ', '.join(sorted(x.unique()))),
    total_deaths = ('Total Deaths', 'sum') if 'Total Deaths' in df_all.columns else (lambda x: np.nan),
    predominant_disaster = ('disastertype', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan) if 'disastertype' in df_all.columns else (lambda x: np.nan),
    all_disaster = ('disastertype', lambda x: ', '.join(sorted(x.unique()))),
    flood_count = ()
).reset_index()

In [ ]:
select_cols = ['grid_id','lat','lon','primary_admin1','unique_locations','predominant_disaster','all_disaster','total_damages','total_deaths','total_disaster_count']
agg_all = agg_all[select_cols]
# agg_all.to_csv('lookup.csv')

### Next we focus on flooding 

In [20]:
#subset to flooding only 
flood_df = merged_df[merged_df['disastertype'].str.lower() == 'flood'].copy()
print("Flood Events Shape:", flood_df.shape)

Flood Events Shape: (4274, 33)


In [83]:
# Central coordinates for each grid cell
flood_df['lat'] = flood_df['lat_grid']
flood_df['lon'] = flood_df['lon_grid']

In [84]:
def compute_duration(row):
    try:
        start_date = pd.Timestamp(year=int(row['Start Year']), month=int(row['Start Month']), day=int(row['Start Day']))
        end_date = pd.Timestamp(year=int(row['End Year']), month=int(row['End Month']), day=int(row['End Day']))
        return (end_date - start_date).days
    except Exception:
        return np.nan

flood_df['duration'] = flood_df.apply(compute_duration, axis=1)


In [85]:
flood_df['duration']

0        12.0
2         7.0
3        46.0
18       16.0
27        0.0
         ... 
82550     4.0
82732     NaN
82734     NaN
82735     NaN
82739     NaN
Name: duration, Length: 4274, dtype: float64

# Aggregate Data by Grid Cell

---
Aggregate Data by Grid Cell (Flood Events Only):

Group the data by grid_id and compute the following parameters:

*total_event_count*: Count of flood events in the grid.

*earliest_event_year*: Minimum of the 'year' column.

*latest_event_year*: Maximum of the 'year' column.

*total_damages*: Sum of "Total Damages, Adjusted ('000 US$)".

*primary_admin1*: Most common administrative region (adm1).

*unique_locations*: Comma-separated list of unique location names.

*avg_duration*: Average duration (in days) of flood events.

#Additional parameters (if available):


*avg_damage_per_flood*: Total_damages divided by total_event_count.

*total_deaths*: Sum of "Total Deaths".

*total_affected*: Sum of "Total Affected".

*reconstruction_costs*: Sum of "Reconstruction Costs, Adjusted ('000 US$)".

*predominant_subtype*: Mode of "Disaster Subtype".

*ofda_response_count*: Count of non-null "OFDA Response".

*predominant_river_basin*: Mode of "River Basin".

*flood_recurrence_interval*: (latest_event_year - earliest_event_year)divided by total_event_count.


In [27]:
def safe_divide(numerator, denominator):
    return numerator / denominator if denominator and denominator != 0 else np.nan

In [86]:
# Aggregate all params
aggregated = flood_df.groupby('grid_id').agg(
    total_event_count = ('disasterno', 'count'),
    earliest_event_year = ('year', 'min'),
    latest_event_year = ('year', 'max'),
    total_damages = ("Total Damages, Adjusted ('000 US$)", 'sum'),
    lat = ('lat', 'first'),
    lon = ('lon', 'first'),
    primary_admin1 = ('adm1', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan),
    # country = ('hist_country', lambda x: x.dropna().iloc[0] if not x.dropna().empty else np.nan),
    unique_locations = ('location', lambda x: ', '.join(sorted(x.unique()))),
    avg_duration = ('duration', 'mean'),
    total_deaths = ('Total Deaths', 'sum') if 'Total Deaths' in flood_df.columns else (lambda x: np.nan),
    total_affected = ('Total Affected', 'sum') if 'Total Affected' in flood_df.columns else (lambda x: np.nan),
    reconstruction_costs = ("Reconstruction Costs, Adjusted ('000 US$)", 'sum') if "Reconstruction Costs, Adjusted ('000 US$)" in flood_df.columns else (lambda x: np.nan),
    predominant_subtype = ('Disaster Subtype', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan) if 'Disaster Subtype' in flood_df.columns else (lambda x: np.nan),
    ofda_response_count = ('OFDA Response', lambda x: x.notnull().sum()) if 'OFDA Response' in flood_df.columns else (lambda x: np.nan),
    predominant_river_basin = ('River Basin', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan) if 'River Basin' in flood_df.columns else (lambda x: np.nan)
).reset_index()

In [87]:
aggregated['total_flood_event'] = aggregated['total_event_count']

# Compute derived parameters:
aggregated['avg_damage_per_flood'] = aggregated.apply(
    lambda row: safe_divide(row['total_damages'], row['total_event_count']),
    axis=1
)
# Flood recurrence formula last-first divided by number of events in the interval
aggregated['flood_recurrence_interval'] = aggregated.apply(
    lambda row: safe_divide(row['latest_event_year'] - row['earliest_event_year'], row['total_event_count']),
    axis=1
)


# Text summary for events in each grid cell

In [88]:
aggregated['event_summary'] = aggregated.apply(
    lambda row: f"{row['total_event_count']} events ({row['total_flood_event']} floods) from {int(row['earliest_event_year'])} to {int(row['latest_event_year'])}, total damages: {row['total_damages'] if not pd.isna(row['total_damages']) else 'N/A'} ('000 US$)",
    axis=1
)


## There is a lot of missing data for $ costs; Replacing 0s with NaNs

In [89]:
# Replace zeros with NaN for cost-related columns in the aggregated DataFrame.
# This assumes that a 0 value for damages or reconstruction costs should be treated as missing data.
cost_columns = ['total_damages', 'reconstruction_costs']
for col in cost_columns:
    aggregated[col] = aggregated[col].replace(0, np.nan)

# Recompute the derived parameter 'avg_damage_per_flood' after the replacement.
aggregated['avg_damage_per_flood'] = aggregated.apply(
    lambda row: safe_divide(row['total_damages'], row['total_event_count']),
    axis=1
)

# Select and Order Final Columns

## We now keep only the desired columns:

 1. grid_id  
 2. central_lat  
 3. central_lon  
 4. total_event_count  
 5. earliest_event_year  
 6. latest_event_year  
 7. total_damages  
 8. primary_admin1  
 9. unique_locations  
 10. avg_duration  
 11. avg_damage_per_flood  
 12. total_deaths  
 13. total_affected  
 14. reconstruction_costs  
 15. predominant_subtype  
 16. ofda_response_count  
 17. predominant_river_basin  
 18. flood_recurrence_interval
 19. event_summary

In [97]:
final_columns = ['grid_id', 'lat', 'lon', 'total_event_count',
                 'earliest_event_year', 'latest_event_year',
                 'total_damages', 'primary_admin1', 'unique_locations', 'avg_duration',
                 'avg_damage_per_flood', 'total_deaths', 'total_affected',
                 'reconstruction_costs', 'predominant_subtype', 'ofda_response_count',
                 'predominant_river_basin', 'flood_recurrence_interval', 'event_summary']

final_df = aggregated[final_columns].copy()


In [95]:
#show percentage of missing data
final_df.isna().sum()/len(final_df)

grid_id                      0.000000
central_lat                  0.000000
central_lon                  0.000000
total_event_count            0.000000
earliest_event_year          0.000000
latest_event_year            0.000000
total_damages                0.507625
primary_admin1               0.000000
country                      0.984205
unique_locations             0.000000
avg_duration                 0.084423
avg_damage_per_flood         0.507625
total_deaths                 0.000000
total_affected               0.000000
reconstruction_costs         0.994009
predominant_subtype          0.168301
ofda_response_count          0.000000
predominant_river_basin      0.581155
flood_recurrence_interval    0.000000
event_summary                0.000000
dtype: float64

In [ ]:
#compute an overall 'hazard score' based on damage, deaths and frequency 
df = final_df.copy()

# Step 1: Min-max normalization (ignoring NaN)
df['count_norm'] = (df['total_event_count'] - df['total_event_count'].min()) / (df['total_event_count'].max() - df['total_event_count'].min())
df['deaths_norm'] = (df['total_deaths'] - df['total_deaths'].min()) / (df['total_deaths'].max() - df['total_deaths'].min())
df['damages_norm'] = (df['total_damages'] - df['total_damages'].min()) / (df['total_damages'].max() - df['total_damages'].min())

# Step 2: Handle missing values
df['count_present'] = ~df['total_event_count'].isna()
df['deaths_present'] = ~df['total_deaths'].isna()
df['damages_present'] = ~df['total_damages'].isna()

# Step 3: Determine dynamic weights
df['num_present'] = df[['count_present', 'deaths_present', 'damages_present']].sum(axis=1)

# Assign weights dynamically
df['count_weight'] = df['count_present'] / df['num_present']
df['deaths_weight'] = df['deaths_present'] / df['num_present']
df['damages_weight'] = df['damages_present'] / df['num_present']

# Fill NA with 0 for Norms 
df['count_norm'] = df['count_norm'].fillna(0)
df['deaths_norm'] = df['deaths_norm'].fillna(0)
df['damages_norm'] = df['damages_norm'].fillna(0)

# Step 4: Compute severity score dynamically
df['severity_score'] = (
df['count_norm'] * df['count_weight'] +
df['deaths_norm'] * df['deaths_weight'] +
df['damages_norm'] * df['damages_weight'])

#drop those cols 
df= df.drop(df.filter(regex='norm|weight|present').columns, axis=1) 

## TO DO: add our predicted risks as well -> TBD 

## TO DO: is min-max normalization the best?? -> @Luca can you think about this, using deaths, loss, count, to compute a severity score 

df.sort_values(by='severity_score', ascending=False).head(50)


,grid_id,lat,lon,total_event_count,earliest_event_year,latest_event_year,total_damages,primary_admin1,unique_locations,avg_duration,avg_damage_per_flood,total_deaths,total_affected,reconstruction_costs,predominant_subtype,ofda_response_count,predominant_river_basin,flood_recurrence_interval,event_summary,severity_score
1038,32_117,32,117,21,1980,2015,121621935.0,Anhui,"Anhui, Anhui, Anhui province, Xiaoxian",24.833333,5.791521e+06,18549.0,1.033919e+09,NaN,Riverine flood,6,43710,1.666667,"21 events (21 floods) from 1980 to 2015, total...",0.760331
1017,31_112,31,112,21,1980,2017,53587610.0,Hubei,"Hubei, Hubei provinces, Enshi, Hubai, Hubai ...",12.578947,2.551791e+06,4172.0,2.555336e+08,NaN,Riverine flood,1,Dalongtan Reservoir on Qingjiang River,1.761905,"21 events (21 floods) from 1980 to 2017, total...",0.415149
446,10_-68,10,-68,7,1994,2011,5440315.0,Distrito Capital,"Carabobo, Caracas",5.714286,7.771879e+05,30194.0,6.109770e+05,NaN,Riverine flood,2,"Guaire, Carmen de Uria, Caugaguita, Upire, Toc...",2.428571,"7 events (7 floods) from 1994 to 2011, total d...",0.414910
906,27_107,27,107,31,1988,2018,22364846.0,Guizhou,"Guizhou, Guizhou Sheng, Guizhou Sheng provi...",10.620690,7.214466e+05,1707.0,1.634767e+08,NaN,Riverine flood,0,Duliujiang,0.967742,"31 events (31 floods) from 1988 to 2018, total...",0.413474
931,28_112,28,112,26,1985,2016,32389279.0,Hunan,"Hunan provinces, Hunan, Hunan provinces, H...",9.739130,1.245742e+06,4048.0,2.157741e+08,NaN,Riverine flood,3,Xiangijiang River,1.192308,"26 events (26 floods) from 1985 to 2016, total...",0.411237
900,26_93,26,93,19,1968,2015,13193169.0,Assam,"Assam, Assam, Assam , Assam province, Assam s...",21.562500,6.943773e+05,3993.0,1.606086e+08,NaN,Riverine flood,1,Brahmaputra,2.473684,"19 events (19 floods) from 1968 to 2015, total...",0.280241
894,26_86,26,86,12,1975,2013,9817307.0,Bihar,"Bihar, Bihar, Bihar state, Darbhanga",12.333333,8.181089e+05,11172.0,1.621085e+08,NaN,Riverine flood,4,"Gandak, Kosi, Sone, Bagmati, Andhawara",3.166667,"12 events (12 floods) from 1975 to 2013, total...",0.272465
1110,34_72,34,72,17,1976,2015,14340690.0,N.W.F.P.,"North-West Frontier, Charsadda, Mardan distri...",8.294118,8.435700e+05,3249.0,2.778529e+07,NaN,Flash flood,2,"Khatayan, Alingar",2.294118,"17 events (17 floods) from 1976 to 2015, total...",0.252950
1771,7_80,7,80,22,1984,2018,625656.0,Colombo,"Colombo, Avissawela, Colombo, Columbo, Kaluta...",5.050000,2.843891e+04,612.0,3.469461e+06,NaN,Riverine flood,1,Kelani,1.545455,"22 events (22 floods) from 1984 to 2018, total...",0.241804
1788,8_100,8,100,20,1975,2016,4183351.0,Nakhon Si Thammarat,"Phatthalung, Hua Sai, Nakhon Si Thammarat, Na...",7.055556,2.091675e+05,1194.0,8.149182e+06,NaN,Riverine flood,4,"Kelantan, Lebir, Golok, Semerak, Tambatan Dira...",2.050000,"20 events (20 floods) from 1975 to 2016, total...",0.235758


# Save data to csv

In [ ]:
final_columns = ['grid_id', 'lat', 'lon', 'total_event_count',
                 'earliest_event_year', 'latest_event_year',
                 'total_damages', 'primary_admin1', 'unique_locations', 'avg_duration',
                 'avg_damage_per_flood', 'total_deaths', 'total_affected',
                 'reconstruction_costs', 'predominant_subtype', 'ofda_response_count',
                 'predominant_river_basin', 'flood_recurrence_interval', 'event_summary']

final_df = df.copy()

final_df.to_csv('/content/drive/MyDrive/mml_flood/UI/flood_grid_summary.csv', index=False)
print("Aggregated flood grid summary data saved to '/content/drive/MyDrive/mml_flood/UI/flood_grid_summary.csv'.")

Aggregated flood grid summary data saved to '/content/drive/MyDrive/mml_flood/UI/flood_grid_summary.csv'.
